# 2. Создание нейронной сети без использования готовых решений

__Автор__: Никита Владимирович Блохин (NVBlokhin@fa.ru)

Финансовый университет, 2020 г. 

In [232]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [233]:
torch.set_warn_always(True)

## 1. Создание нейронов и полносвязных слоев

1.1. Используя операции над матрицами и векторами из библиотеки `torch`, реализовать нейрон с заданными весами `weights` и `bias`. Прогнать вектор `inputs` через нейрон и вывести результат. 

In [234]:
class Neuron:

    def __init__(self, weights: torch.Tensor, bias: torch.Tensor):
        self.weights = weights
        self.bias = bias

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        return torch.sum(inputs * self.weights) + self.bias

In [235]:
inputs = torch.tensor([1.0, 2.0, 3.0, 4.0])
weights = torch.tensor([-0.2, 0.3, -0.5, 0.7])
bias = torch.tensor(3.14)

In [236]:
neuron = Neuron(weights, bias)
print(f'Neuron.forward: {neuron.forward(inputs)}')
print(f'functional.linear: {F.linear(inputs, weights, bias)}')

Neuron.forward: 4.840000152587891
functional.linear: 4.840000152587891


1.2 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать полносвязный слой с заданными весами `weights` и `biases`. Прогнать вектор `inputs` через слой и вывести результат. 

In [237]:
class Linear:

    def __init__(self, weights: torch.Tensor, biases: torch.Tensor):
        self.weights = weights
        self.biases = biases

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        return torch.matmul(inputs, self.weights.T) + self.biases

In [238]:
inputs = torch.tensor([1.0, 2.0, 3.0, 4.0])
weights = torch.tensor([[-0.2, 0.3, -0.5, 0.7],
                        [0.5, -0.91, 0.26, -0.5],
                        [-0.26, -0.27, 0.17, 0.87]])  # убрал .T

biases = torch.tensor([3.14, 2.71, 7.2])

In [239]:
m = Linear(weights, biases)
print(f'Linear.forward:\n{m.forward(inputs)}')
print(f'\nfunctional.linear:\n{F.linear(inputs, weights, biases)}')

Linear.forward:
tensor([ 4.8400,  0.1700, 10.3900])

functional.linear:
tensor([ 4.8400,  0.1700, 10.3900])


1.3 Реализовать полносвязный слой из __2.1.2__ таким образом, чтобы он мог принимать на вход матрицу (батч) с данными. Продемонстрировать работу.
Результатом прогона сквозь слой должна быть матрица размера `batch_size` x `n_neurons`.


In [240]:
inputs = torch.tensor([[1, 2, 3, 2.5],
                       [2, 5, -1, 2],
                       [-1.5, 2.7, 3.3, -0.8]])

In [241]:
m = Linear(weights, biases)
print(f'Linear.forward:\n{m.forward(inputs)}')
print(f'\nfunctional.linear:\n{F.linear(inputs, weights, biases)}')

Linear.forward:
tensor([[ 3.7900,  0.9200,  9.0850],
        [ 6.1400, -2.1000,  6.9000],
        [ 2.0400,  0.7610,  6.7260]])

functional.linear:
tensor([[ 3.7900,  0.9200,  9.0850],
        [ 6.1400, -2.1000,  6.9000],
        [ 2.0400,  0.7610,  6.7260]])


1.4 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать полносвязный слой из `n_neurons` нейронов с `n_features` весами у каждого нейрона (инициализируются из стандартного нормального распределения). Прогнать вектор `inputs` через слой и вывести результат. Результатом прогона сквозь слой должна быть матрица размера `batch_size` x `n_neurons`.

In [242]:
class Linear:

    def __init__(self, in_features: int, out_features: int):
        self.in_features = in_features
        self.out_features = out_features

        self.weights = torch.randn(out_features, in_features, requires_grad=True)
        self.biases = torch.randn(out_features, requires_grad=True)

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        return torch.matmul(inputs, self.weights.T) + self.biases

In [243]:
torch.manual_seed(0)

m = Linear(4, 3)
m.forward(inputs)

tensor([[ -5.0178,   0.5240,  -3.9319],
        [  4.0738,  -6.7887,  -3.5657],
        [-11.6052,  -0.3882,  -3.1959]], grad_fn=<AddBackward0>)

In [244]:
# проверка
nn_m = nn.Linear(4, 3)
nn_m.weight = nn.Parameter(m.weights)
nn_m.bias = nn.Parameter(m.biases)
nn_m.forward(inputs)

tensor([[ -5.0178,   0.5240,  -3.9319],
        [  4.0738,  -6.7887,  -3.5657],
        [-11.6052,  -0.3882,  -3.1959]], grad_fn=<AddmmBackward0>)

1.5 Используя решение из __1.4__, создать 2 полносвязных слоя и пропустить матрицу `inputs` последовательно через эти два слоя. Количество нейронов в первом слое выбрать произвольно, количество нейронов во втором слое выбрать так, чтобы результатом прогона являлась матрица (3x7).

In [245]:
class NeuralNet:

    def __init__(self):
        self.input = Linear(4, 3)
        self.output = Linear(3, 7)

    def forward(self, inputs) -> torch.Tensor:
        return self.output.forward(self.input.forward(inputs))

In [246]:
inputs = torch.tensor([[1, 2, 3, 2.5],
                       [2, 5, -1, 2],
                       [-1.5, 2.7, 3.3, -0.8]])

In [247]:
torch.manual_seed(0)

m = NeuralNet()
m.forward(inputs)

tensor([[  5.1553,   1.1916,  -4.7817,   3.6611,  -9.6546,   0.1692,   5.8099],
        [ -0.8347,   0.0913,   3.1227,   3.5542,   5.4400,   4.9157,   8.2533],
        [  8.8057,  -1.1342,  -7.8488,   6.7415, -11.1044,   5.1500,   5.6152]],
       grad_fn=<AddBackward0>)

## 2. Создание функций активации

In [248]:
def randn(*size: int, seed: int = 0) -> torch.Tensor:
    torch.manual_seed(seed)
    return torch.randn(*size)

2.1 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать функцию активации ReLU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/f4353f4e3e484130504049599d2e7b040793e1eb)

Создать матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверить работоспособность функции активации.

In [249]:
class ReLU:

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        return torch.maximum(inputs, torch.tensor(0))

In [250]:
inputs = randn(4, 3)
inputs

tensor([[ 1.5410, -0.2934, -2.1788],
        [ 0.5684, -1.0845, -1.3986],
        [ 0.4033,  0.8380, -0.7193],
        [-0.4033, -0.5966,  0.1820]])

In [251]:
relu = ReLU()
nn_relu = nn.ReLU()
print(f'ReLU:\n{relu.forward(inputs)}')
print(f'\nnn.ReLU:\n{nn_relu.forward(inputs)}')

ReLU:
tensor([[1.5410, 0.0000, 0.0000],
        [0.5684, 0.0000, 0.0000],
        [0.4033, 0.8380, 0.0000],
        [0.0000, 0.0000, 0.1820]])

nn.ReLU:
tensor([[1.5410, 0.0000, 0.0000],
        [0.5684, 0.0000, 0.0000],
        [0.4033, 0.8380, 0.0000],
        [0.0000, 0.0000, 0.1820]])


2.2 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать функцию активации softmax:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/6d7500d980c313da83e4117da701bf7c8f1982f5)

Создать матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверить работоспособность функции активации. Строки матрицы трактовать как выходы линейного слоя некоторого классификатора для 4 различных примеров.

In [252]:
class Softmax:

    def __init__(self, dim: int = 0):
        assert dim == 0 or dim == 1
        self.dim = dim

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        exp = torch.exp(inputs)
        return exp / torch.sum(exp, dim=self.dim).unsqueeze(self.dim)

In [253]:
inputs = randn(4, 3)
inputs

tensor([[ 1.5410, -0.2934, -2.1788],
        [ 0.5684, -1.0845, -1.3986],
        [ 0.4033,  0.8380, -0.7193],
        [-0.4033, -0.5966,  0.1820]])

In [254]:
softmax = Softmax(dim=1)
nn_softmax = nn.Softmax(dim=1)
print(f'Softmax:\n{softmax.forward(inputs)}')
print(f'\nnn.Softmax:\n{nn_softmax.forward(inputs)}')

Softmax:
tensor([[0.8446, 0.1349, 0.0205],
        [0.7511, 0.1438, 0.1051],
        [0.3484, 0.5382, 0.1134],
        [0.2762, 0.2277, 0.4961]])

nn.Softmax:
tensor([[0.8446, 0.1349, 0.0205],
        [0.7511, 0.1438, 0.1051],
        [0.3484, 0.5382, 0.1134],
        [0.2762, 0.2277, 0.4961]])


2.3 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать функцию активации ELU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/eb23becd37c3602c4838e53f532163279192e4fd)

Создать матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверить работоспособность функции активации.

In [255]:
class ELU:

    def __init__(self, alpha: float = 1):
        self.alpha = alpha

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        return torch.where(inputs > 0, inputs, self.alpha * (torch.exp(inputs) - 1))

In [256]:
inputs = randn(4, 3)
inputs

tensor([[ 1.5410, -0.2934, -2.1788],
        [ 0.5684, -1.0845, -1.3986],
        [ 0.4033,  0.8380, -0.7193],
        [-0.4033, -0.5966,  0.1820]])

In [257]:
elu = ELU(alpha=1.409)
nn_elu = nn.ELU(alpha=1.409)
print(f'ELU:\n{elu.forward(inputs)}')
print(f'\nnn.ELU:\n{nn_elu.forward(inputs)}')

ELU:
tensor([[ 1.5410, -0.3583, -1.2495],
        [ 0.5684, -0.9327, -1.0611],
        [ 0.4033,  0.8380, -0.7227],
        [-0.4677, -0.6331,  0.1820]])

nn.ELU:
tensor([[ 1.5410, -0.3583, -1.2495],
        [ 0.5684, -0.9327, -1.0611],
        [ 0.4033,  0.8380, -0.7227],
        [-0.4677, -0.6331,  0.1820]])


## 3. Создание функции потерь

3.1 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать функцию потерь MSE:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/e258221518869aa1c6561bb75b99476c4734108e)

Создать полносвязный слой с 1 нейроном, прогнать через него батч `inputs` и посчитать значение MSE, трактуя вектор `y` как вектор правильных ответов.

In [258]:
class MSELoss:

    def forward(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        return torch.mean((y_true - y_pred) ** 2)

In [259]:
inputs = torch.tensor([[1, 2, 3, 2.5],
                       [2, 5, -1, 2],
                       [-1.5, 2.7, 3.3, -0.8]])

y = torch.tensor([2, 3, 4]).unsqueeze(1)

In [260]:
torch.manual_seed(0)

layer = Linear(4, 1)
y_pred = layer.forward(inputs)

In [261]:
mse = MSELoss()
nn_mse = nn.MSELoss()
print(f'MSELoss:\n{mse.forward(y_pred, y)}')
print(f'\nnn.MSELoss:\n{nn_mse.forward(y_pred, y)}')

MSELoss:
101.30004119873047

nn.MSELoss:
101.30004119873047


3.2 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать функцию потерь Categorical Cross-Entropy:

<img src="https://i.ibb.co/93gy1dN/Screenshot-9.png" width="200">

Создать полносвязный слой с 3 нейронами и прогнать через него батч `inputs`. Полученный результат пропустить через функцию активации softmax. Посчитать значение CCE, трактуя вектор `y` как вектор правильных ответов.

In [262]:
class CategoricalCrossEntropyLoss:

    def forward(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        return -torch.sum(y_true * torch.log(y_pred), dim=1)

In [263]:
inputs = torch.tensor([[1, 2, 3, 2.5],
                       [2, 5, -1, 2],
                       [-1.5, 2.7, 3.3, -0.8]])

y = torch.tensor([1, 0, 0])

In [264]:
torch.manual_seed(0)

layer = Linear(4, 3)
softmax = Softmax()
y_pred = softmax.forward(layer.forward(inputs))

In [265]:
cce = CategoricalCrossEntropyLoss()
cce.forward(y_pred, y)

tensor([9.0918e+00, 1.1272e-04, 1.5679e+01], grad_fn=<NegBackward0>)

3.3 Модифицировать 2.3.1, добавив L2-регуляризацию.

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/d92ca2429275bfdc0474523babbafe014ca8b580)


In [266]:
class MSELossL2:

    def __init__(self, lambda_: float, weights: torch.Tensor):
        self.lambda_ = lambda_
        self.weights = weights

    def data_loss(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        return torch.sum((y_true - y_pred) ** 2)

    def reg_loss(self) -> torch.Tensor:
        return self.lambda_ * torch.sum(self.weights ** 2)

    def forward(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        return self.data_loss(y_pred, y_true) + self.reg_loss()

In [267]:
inputs = torch.tensor([[1, 2, 3, 2.5],
                       [2, 5, -1, 2],
                       [-1.5, 2.7, 3.3, -0.8]])

y = torch.tensor([2, 3, 4]).unsqueeze(1)

In [268]:
torch.manual_seed(0)

layer = Linear(4, 1)
y_pred = layer.forward(inputs)

In [269]:
mse_l2 = MSELossL2(lambda_=1.409, weights=layer.weights)
mse_l2.forward(y_pred, y)

tensor(314.5113, grad_fn=<AddBackward0>)

## 4. Обратное распространение ошибки

4.1 Используя один нейрон и SGD (1 пример за шаг), решите задачу регрессии

In [270]:
from sklearn.datasets import make_regression

X, y, coef = make_regression(n_features=4, n_informative=4, coef=True, bias=0.5, random_state=0)
# X.dtype == float64 - нет смысла использовать torch.from_numpy
# т.к. последующее приведение типов Tensor.type(torch.float32) приведет к копированию данных
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

[Граф вычислений для этой задачи](https://i.ibb.co/2dhDxZx/photo-2021-02-15-17-18-04.jpg)

4.1.1 Модифицируйте класс `MSELoss` из __2.3.1__, реализовав расчет производной относительно предыдущего слоя


In [271]:
class MSELoss:

    def forward(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        return (y_pred - y_true) ** 2  # когда MSE (mean squared error) без mean

    def backward(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        # здравая мысль - похоже на производную MSE, только mean потерялась,
        # т.к. обработка по одному ответу за шаг
        return 2 * (y_pred - y_true)  # df/dc

4.1.2. Модифицируйте класс `Neuron` из __2.1.1__:

  1) Сделайте так, чтобы веса нейрона инициализировались из стандартного нормального распределения

  2) Реализуйте расчет градиента относительно весов `weights` и `bias`

In [272]:
class Neuron:

    def __init__(self, in_features: int):
        self.in_features = in_features
        self.weights = torch.randn(self.in_features)
        self.bias = torch.randn(1)

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        return torch.sum(inputs * self.weights) + self.bias

    def backward(self, dvalue: torch.Tensor, inputs: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        dweights = dvalue * 1  # df/dw
        dbias = dvalue * 1  # df/db
        dinputs = dbias * self.weights  # df/dx
        return dvalue * inputs, dvalue  # считаем градиент (типа на предшествующем слое)

4.1.3 Допишите цикл для настройки весов нейрона

[SGD](https://ru.wikipedia.org/wiki/%D0%A1%D1%82%D0%BE%D1%85%D0%B0%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9_%D0%B3%D1%80%D0%B0%D0%B4%D0%B8%D0%B5%D0%BD%D1%82%D0%BD%D1%8B%D0%B9_%D1%81%D0%BF%D1%83%D1%81%D0%BA)

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/dda3670f8a8996a0d3bf80856bb4a166cc8db6d4)

In [273]:
def pretty_train_log(epoch: int, **kwargs) -> None:
    params = '\n\t'.join([f'{k} = {v}' for k, v in kwargs.items()])
    print(f'epoch {epoch:03} |' + '-' * 50 + f'\n\t{params}\n')

In [274]:
# работает как часы 🕓...
torch.manual_seed(0)

n_inputs = X.size(1)  # размерность элемента выборки
learning_rate = 0.1  #  скорость обучения
n_epoch = 100  #  количество эпох

y_true = y.unsqueeze(1)

neuron = Neuron(n_inputs)
loss = MSELoss()

for epoch in range(1, n_epoch + 1):
    for i, (x_example, y_example) in enumerate(zip(X, y_true), start=1):
        # forward pass
        y_pred = neuron.forward(x_example)  # прогон через нейрон
        curr_loss = loss.forward(y_pred, y_example)  # прогон через функцию потерь

        # backprop
        # вызов методов backward
        dw, db = neuron.backward(loss.backward(y_pred, y_example), x_example)
        # обратите внимание на последовательность вызовов: от конца к началу

        # шаг оптимизации для весов (weights и bias) нейрона
        neuron.weights -= learning_rate * dw
        neuron.bias -= learning_rate * db

        if (epoch - 1) % 20 == 0 and (i - 1) % 30 == 0:
            pretty_train_log(epoch, i=i, weights=neuron.weights, dw=dw, bias=neuron.bias, db=db, loss=curr_loss)

neuron.weights, coef

epoch 001 |--------------------------------------------------
	i = 1
	weights = tensor([ 5.8937,  3.4480,  8.5208, -6.0718])
	dw = tensor([ -43.5271,  -37.4140, -106.9956,   66.4020])
	bias = tensor([-9.6251])
	db = tensor([85.4055])
	loss = tensor([1823.5259])

epoch 001 |--------------------------------------------------
	i = 31
	weights = tensor([21.6376, 36.3937, 65.8168, 88.4170])
	dw = tensor([9.4374, 2.5177, 7.3150, 2.5319])
	bias = tensor([1.1639])
	db = tensor([7.9437])
	loss = tensor([15.7756])

epoch 001 |--------------------------------------------------
	i = 61
	weights = tensor([20.5037, 34.1982, 67.6044, 87.9179])
	dw = tensor([ 0.0004,  0.0003, -0.0005,  0.0003])
	bias = tensor([0.4687])
	db = tensor([0.0006])
	loss = tensor([1.0514e-07])

epoch 001 |--------------------------------------------------
	i = 91
	weights = tensor([20.4878, 34.1696, 67.6197, 87.9236])
	dw = tensor([-0.0058,  0.0145,  0.0002,  0.0066])
	bias = tensor([0.5022])
	db = tensor([-0.0090])
	loss = 

(tensor([20.4924, 34.1698, 67.6242, 87.9235]),
 array([20.4923687 , 34.16981149, 67.62424823, 87.9234763 ]))

Судя по выводу задача регрессии была решена еще в конце 1-ой эпохи (первобытности).
Зачем тогда пакетный градиентный спуск, если этот работает быстрее?

Зачем мы это все делали и почему называем обратным распространением, если уже знаем всю производную?

**Проверка**

Сделаем то же самое, но пусть производную посчитает pytorch (честно, будто не знаем производную сложной функции)

In [275]:
class MSELoss:

    def __init__(self):
        self._last_forward: torch.Tensor | None = None

    def forward(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        self._last_forward = (y_pred - y_true) ** 2
        return self._last_forward

    def backward(self) -> None:
        self._last_forward.backward()


class Neuron:

    def __init__(self, in_features: int):
        self.in_features = in_features
        self.weights = torch.randn(self.in_features, requires_grad=True)
        self.bias = torch.randn(1, requires_grad=True)

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        return torch.sum(inputs * self.weights) + self.bias

    def backward(self) -> tuple[torch.Tensor, torch.Tensor]:
        dw = self.weights.grad.clone().detach()
        db = self.bias.grad.clone().detach()
        self.weights.grad.zero_()
        self.bias.grad.zero_()
        return dw, db

    def update_weights(self, lr: float, dw: torch.Tensor, db: torch.Tensor) -> None:
        with torch.no_grad():
            self.weights -= lr * dw
            self.bias -= lr * db

In [276]:
torch.manual_seed(0)

y_true = y.unsqueeze(1)

neuron = Neuron(X.size(1))
mse = MSELoss()

learning_rate = 0.1

epochs = 100
for epoch in range(1, epochs + 1):
    for i, (x_example, y_example) in enumerate(zip(X, y_true), start=1):
        # feed forward
        y_pred = neuron.forward(x_example)
        loss = mse.forward(y_pred, y_example)

        # backward
        mse.backward()
        dw, db = neuron.backward()

        # optimize step
        neuron.update_weights(learning_rate, dw, db)

        if (epoch - 1) % 20 == 0 and (i - 1) % 30 == 0:
            pretty_train_log(epoch, i=i, weights=neuron.weights, dw=dw, bias=neuron.bias, db=db, loss=loss)

neuron.weights, coef

epoch 001 |--------------------------------------------------
	i = 1
	weights = tensor([ 5.8937,  3.4480,  8.5208, -6.0718], requires_grad=True)
	dw = tensor([ -43.5271,  -37.4140, -106.9956,   66.4020])
	bias = tensor([-9.6251], requires_grad=True)
	db = tensor([85.4055])
	loss = tensor([1823.5259], grad_fn=<PowBackward0>)

epoch 001 |--------------------------------------------------
	i = 31
	weights = tensor([21.6376, 36.3937, 65.8168, 88.4170], requires_grad=True)
	dw = tensor([9.4374, 2.5177, 7.3150, 2.5319])
	bias = tensor([1.1639], requires_grad=True)
	db = tensor([7.9437])
	loss = tensor([15.7756], grad_fn=<PowBackward0>)

epoch 001 |--------------------------------------------------
	i = 61
	weights = tensor([20.5037, 34.1982, 67.6044, 87.9179], requires_grad=True)
	dw = tensor([ 0.0004,  0.0003, -0.0005,  0.0003])
	bias = tensor([0.4687], requires_grad=True)
	db = tensor([0.0006])
	loss = tensor([1.0514e-07], grad_fn=<PowBackward0>)

epoch 001 |-------------------------------

(tensor([20.4924, 34.1698, 67.6242, 87.9235], requires_grad=True),
 array([20.4923687 , 34.16981149, 67.62424823, 87.9234763 ]))

In [277]:
class MSELoss:

    def __init__(self):
        self._last_forward: torch.Tensor | None = None

    def forward(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        self._last_forward = torch.mean((y_pred - y_true) ** 2)
        return self._last_forward

    def backward(self) -> None:
        self._last_forward.backward()

In [278]:
class Neuron:

    def __init__(self, in_features: int):
        self.in_features = in_features
        self.weights = torch.randn(self.in_features, requires_grad=True)
        self.bias = torch.randn(1, requires_grad=True)

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        return torch.sum(inputs * self.weights) + self.bias

    def backward(self) -> tuple[torch.Tensor, torch.Tensor]:
        dw = self.weights.grad.clone().detach()
        db = self.bias.grad.clone().detach()
        self.weights.grad.zero_()
        self.bias.grad.zero_()
        return dw, db

    def update_weights(self, lr: float, dw: torch.Tensor, db: torch.Tensor) -> None:
        with torch.no_grad():
            self.weights -= lr * dw
            self.bias -= lr * db

In [279]:
torch.manual_seed(0)

y_true = y.unsqueeze(1)

neuron = Neuron(X.size(1))
mse = MSELoss()

learning_rate = 0.1

epochs = 100
for epoch in range(1, epochs + 1):
    y_pred = torch.empty(y_true.size())
    for i, x in enumerate(X):
        y_pred[i, 0] = neuron.forward(x)

    loss = mse.forward(y_pred, y_true)

    mse.backward()
    dw, db = neuron.backward()
    neuron.update_weights(learning_rate, dw, db)

    if epoch % 10 == 0:
        print(f'epoch {epoch}:\n'
              f'\tdw = {dw}\n'
              f'\tw = {neuron.weights}\n'
              f'\tdb = {db}\n'
              f'\tb = {neuron.bias}\n'
              f'\tloss = {loss}\n')

epoch 10:
	dw = tensor([  0.0946,  -5.0925, -19.5831, -23.7725])
	w = tensor([22.8361, 35.1808, 56.8774, 77.7312], requires_grad=True)
	db = tensor([-1.1937])
	b = tensor([-0.3279], requires_grad=True)
	loss = 273.2061462402344

epoch 20:
	dw = tensor([ 1.2022,  1.0444, -3.6096, -2.9211])
	w = tensor([21.3865, 35.2819, 65.2723, 86.5805], requires_grad=True)
	db = tensor([-0.2879])
	b = tensor([0.2734], requires_grad=True)
	loss = 8.566176414489746

epoch 30:
	dw = tensor([ 0.3433,  0.4741, -0.8571, -0.4070])
	w = tensor([20.7266, 34.5308, 67.0295, 87.7218], requires_grad=True)
	db = tensor([-0.0849])
	b = tensor([0.4255], requires_grad=True)
	loss = 0.48739707469940186

epoch 40:
	dw = tensor([ 0.0871,  0.1401, -0.2223, -0.0644])
	w = tensor([20.5520, 34.2695, 67.4672, 87.8893], requires_grad=True)
	db = tensor([-0.0294])
	b = tensor([0.4750], requires_grad=True)
	loss = 0.032591093331575394

epoch 50:
	dw = tensor([ 0.0222,  0.0377, -0.0591, -0.0114])
	w = tensor([20.5077, 34.1962, 67

Используем слой из 1 нейрона и почти все отдаем pytorch кроме обновления весов

In [280]:
torch.manual_seed(0)

Y = y.unsqueeze(1)

layer = Linear(X.size(1), 1)
mse = MSELoss()

learning_rate = 0.01409

epochs = 500
for epoch in range(1, epochs + 1):
    y_pred = layer.forward(X)
    loss = mse.forward(y_pred, Y)

    loss.backward()
    with torch.no_grad():
        layer.weights -= learning_rate * layer.weights.grad
        layer.biases -= learning_rate * layer.biases.grad

    if epoch % 50 == 0:
        print(f'epoch {epoch}:\n'
              f'\tdw = {layer.weights.grad}\n'
              f'\tw = {layer.weights}\n'
              f'\tdb = {layer.biases.grad}\n'
              f'\tb = {layer.biases}\n'
              f'\tloss = {loss}\n')

    layer.weights.grad.zero_()
    layer.biases.grad.zero_()


epoch 50:
	dw = tensor([[ -6.0558, -15.0842, -33.4915, -42.9164]])
	w = tensor([[21.5726, 31.4347, 47.8225, 66.2012]], requires_grad=True)
	db = tensor([-1.3325])
	b = tensor([-0.7389], requires_grad=True)
	loss = 838.6507568359375

epoch 100:
	dw = tensor([[  1.1853,  -0.8189,  -9.8127, -10.7610]])
	w = tensor([[22.2123, 35.4245, 61.0621, 82.3018]], requires_grad=True)
	db = tensor([-0.6651])
	b = tensor([-0.0315], requires_grad=True)
	loss = 64.63803100585938

epoch 150:
	dw = tensor([[ 0.9964,  0.7865, -3.3318, -2.8093]])
	w = tensor([[21.3588, 35.1799, 65.2005, 86.4030]], requires_grad=True)
	db = tensor([-0.2642])
	b = tensor([0.2715], requires_grad=True)
	loss = 7.178253650665283

epoch 200:
	dw = tensor([[ 0.4676,  0.5566, -1.2485, -0.7666]])
	w = tensor([[20.8615, 34.6863, 66.6720, 87.4926]], requires_grad=True)
	db = tensor([-0.1130])
	b = tensor([0.3947], requires_grad=True)
	loss = 1.0145002603530884

epoch 250:
	dw = tensor([[ 0.1950,  0.2764, -0.4942, -0.2191]])
	w = tenso

4.2 Решите задачу 2.4.1, используя пакетный градиентный спуск

Вычисления для этой задачи: 
[1](https://i.ibb.co/rmtQT6P/photo-2021-02-15-18-00-43.jpg)
[2](https://i.ibb.co/NmCFVnQ/photo-2021-02-15-18-01-17.jpg)

4.2.1 Модифицируйте класс `MSELoss` из __3.1__, реализовав расчет производной относительно предыдущего слоя с учетом того, что теперь работа ведется с батчами, а не с индивидуальными примерами
 

In [281]:
# использование mean в методах вызывает доверие к имени класса
class MSELoss:

    def forward(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        return torch.mean((y_pred - y_true) ** 2)

    def backward(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        return 2 * (y_pred - y_true)

4.2.2. Модифицируйте класс `Neuron` из __4.1.2__:

  1) Реализуйте метод `forward` таким образом, чтобы он мог принимать на вход матрицу (батч) с данными. 

  2) Реализуйте расчет градиента относительно весов `weights` и `bias` с учетом того, что теперь работа ведется с батчами, а не с индивидуальными примерами

In [282]:
# а это точно еще нейрон, а не полносвязный слой из одного нейрона с in_features входами?
class Neuron:

    def __init__(self, in_features: int):
        self.in_features = in_features
        self.weights = torch.randn(1, in_features)
        self.bias = torch.randn(1)

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        return torch.matmul(inputs, self.weights.T) + self.bias

    def backward(self, dvalue: torch.Tensor, inputs: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        return torch.mean(dvalue * inputs, dim=0), torch.mean(dvalue)

4.2.3 Допишите цикл для настройки весов нейрона

In [283]:
torch.manual_seed(0)

y_true = y.unsqueeze(1)
neuron = Neuron(X.size(1))
mse = MSELoss()

learning_rate = 0.1
epochs = 100

for epoch in range(1, epochs + 1):
    y_pred = neuron.forward(X)
    loss = mse.forward(y_pred, y_true)

    dw, db = neuron.backward(mse.backward(y_pred, y_true), X)

    neuron.weights -= learning_rate * dw
    neuron.bias -= learning_rate * db

    if epoch % 10 == 0:
        pretty_train_log(epoch, weights=neuron.weights, dw=dw, bias=neuron.bias, db=db, loss=loss)

neuron.weights, coef

epoch 010 |--------------------------------------------------
	weights = tensor([[22.8361, 35.1808, 56.8774, 77.7312]])
	dw = tensor([  0.0946,  -5.0925, -19.5831, -23.7725])
	bias = tensor([-0.3279])
	db = -1.1936815977096558
	loss = 273.2061462402344

epoch 020 |--------------------------------------------------
	weights = tensor([[21.3865, 35.2819, 65.2723, 86.5805]])
	dw = tensor([ 1.2022,  1.0444, -3.6096, -2.9211])
	bias = tensor([0.2734])
	db = -0.28787899017333984
	loss = 8.566173553466797

epoch 030 |--------------------------------------------------
	weights = tensor([[20.7266, 34.5308, 67.0295, 87.7218]])
	dw = tensor([ 0.3433,  0.4741, -0.8571, -0.4070])
	bias = tensor([0.4255])
	db = -0.08487050980329514
	loss = 0.4873894155025482

epoch 040 |--------------------------------------------------
	weights = tensor([[20.5520, 34.2695, 67.4672, 87.8893]])
	dw = tensor([ 0.0871,  0.1401, -0.2223, -0.0644])
	bias = tensor([0.4750])
	db = -0.029402075335383415
	loss = 0.03259115293

(tensor([[20.4924, 34.1698, 67.6242, 87.9235]]),
 array([20.4923687 , 34.16981149, 67.62424823, 87.9234763 ]))

4.3 Используя один полносвязный слой и пакетный градиентный спуск, решите задачу регрессии из __2.4.1__

4.3.1 Модифицируйте класс `Linear` из __1.4__. ([вычисление градиентов](https://i.ibb.co/kgVR6m6/photo-2021-02-15-21-30-28.jpg))

In [284]:
# подозрительно 🧐 похоже на предыдущий нейрон (просто совпадение)
class Linear:

    def __init__(self, in_features: int, out_features: int):
        self.in_features = in_features
        self.out_features = out_features

        self.weights = torch.randn(out_features, in_features)
        self.biases = torch.randn(out_features)

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        return torch.matmul(inputs, self.weights.T) + self.biases

    def backward(self, dvalue: torch.Tensor, inputs: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        return torch.mean(dvalue * inputs, dim=0), torch.mean(dvalue)

4.3.2 Создайте слой с одним нейроном. Используя класс MSELoss из 2.4.2, убедитесь, что модель обучается

In [285]:
torch.manual_seed(0)

y_true = y.unsqueeze(1)
layer = Linear(X.size(1), 1)
mse = MSELoss()

learning_rate = 0.1
epochs = 100

for epoch in range(1, epochs + 1):
    y_pred = layer.forward(X)
    loss = mse.forward(y_pred, y_true)

    dw, db = layer.backward(mse.backward(y_pred, y_true), X)

    layer.weights -= learning_rate * dw
    layer.biases -= learning_rate * db

    if epoch % 10 == 0:
        pretty_train_log(epoch, weights=layer.weights, dw=dw, biases=layer.biases, db=db, loss=loss)

neuron.weights, coef

epoch 010 |--------------------------------------------------
	weights = tensor([[22.8361, 35.1808, 56.8774, 77.7312]])
	dw = tensor([  0.0946,  -5.0925, -19.5831, -23.7725])
	biases = tensor([-0.3279])
	db = -1.1936815977096558
	loss = 273.2061462402344

epoch 020 |--------------------------------------------------
	weights = tensor([[21.3865, 35.2819, 65.2723, 86.5805]])
	dw = tensor([ 1.2022,  1.0444, -3.6096, -2.9211])
	biases = tensor([0.2734])
	db = -0.28787899017333984
	loss = 8.566173553466797

epoch 030 |--------------------------------------------------
	weights = tensor([[20.7266, 34.5308, 67.0295, 87.7218]])
	dw = tensor([ 0.3433,  0.4741, -0.8571, -0.4070])
	biases = tensor([0.4255])
	db = -0.08487050980329514
	loss = 0.4873894155025482

epoch 040 |--------------------------------------------------
	weights = tensor([[20.5520, 34.2695, 67.4672, 87.8893]])
	dw = tensor([ 0.0871,  0.1401, -0.2223, -0.0644])
	biases = tensor([0.4750])
	db = -0.029402075335383415
	loss = 0.032

(tensor([[20.4924, 34.1698, 67.6242, 87.9235]]),
 array([20.4923687 , 34.16981149, 67.62424823, 87.9234763 ]))

**Проверка**

А pytorch точно правильно работает?

In [286]:
torch.manual_seed(0)

weights = torch.randn(1, 4)
bias = torch.randn(1)

y_true = y.unsqueeze(1)
layer = nn.Linear(X.size(1), 1)
layer.weight = nn.Parameter(weights, requires_grad=True)
layer.bias = nn.Parameter(bias, requires_grad=True)
mse = nn.MSELoss()

learning_rate = 0.1
optimizer = torch.optim.SGD([layer.weight, layer.bias], lr=learning_rate)
epochs = 100

for epoch in range(1, epochs + 1):
    y_pred = layer.forward(X)
    loss = mse.forward(y_pred, y_true)

    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        pretty_train_log(epoch, weight=layer.weight, bias=layer.bias, loss=loss)

    optimizer.zero_grad()

neuron.weights, coef

epoch 010 |--------------------------------------------------
	weight = Parameter containing:
tensor([[22.8361, 35.1808, 56.8774, 77.7312]], requires_grad=True)
	bias = Parameter containing:
tensor([-0.3279], requires_grad=True)
	loss = 273.20623779296875

epoch 020 |--------------------------------------------------
	weight = Parameter containing:
tensor([[21.3865, 35.2819, 65.2723, 86.5805]], requires_grad=True)
	bias = Parameter containing:
tensor([0.2734], requires_grad=True)
	loss = 8.566173553466797

epoch 030 |--------------------------------------------------
	weight = Parameter containing:
tensor([[20.7266, 34.5308, 67.0295, 87.7218]], requires_grad=True)
	bias = Parameter containing:
tensor([0.4255], requires_grad=True)
	loss = 0.4873894155025482

epoch 040 |--------------------------------------------------
	weight = Parameter containing:
tensor([[20.5520, 34.2695, 67.4672, 87.8893]], requires_grad=True)
	bias = Parameter containing:
tensor([0.4750], requires_grad=True)
	los

(tensor([[20.4924, 34.1698, 67.6242, 87.9235]]),
 array([20.4923687 , 34.16981149, 67.62424823, 87.9234763 ]))

4.4 Используя наработки из 2.4, создайте нейросеть и решите задачу регрессии.

Предлагаемая архитектура: 
1. Полносвязный слой с 10 нейронами
2. Активация ReLU
3. Полносвязный слой с 1 нейроном

In [287]:
X = torch.linspace(-1, 1, 100).view(-1, 1)
y = X.pow(2) + 0.2 * torch.rand(X.size())

In [288]:
class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = inputs.clip(min=0)
        return self.output

    def backward(self, dvalues):
        self.dinputs = dvalues.clone()
        self.dinputs[self.inputs <= 0] = 0

In [289]:
# создание компонентов сети
# fc1 = 
# relu1 = 
# fc2 = 

loss = MSELoss()
lr = 0.02

ys = []
for epoch in range(2001):
    # <forward pass>
    # fc1 > relu1 > fc2 > loss

    data_loss =  # <прогон через функцию потерь>

    if epoch % 200 == 0:
        print(f'epoch {epoch} mean loss {data_loss}')
        ys.append(out)

    # <backprop>
    # loss > fc2 > relu1 > fc1

    # <шаг оптимизации для fc1>

    # <шаг оптимизации для fc2>


SyntaxError: invalid syntax (1143161818.py, line 14)

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(len(ys), 1, figsize=(10, 40))
for ax, y_ in zip(axs, ys):
    ax.scatter(X.numpy(), y.numpy(), color="orange")
    ax.plot(X.numpy(), y_.numpy(), 'g-', lw=3)
    ax.set_xlim(-1.05, 1.5)
    ax.set_ylim(-0.25, 1.25)